In [1]:
from instaloader import Instaloader, Profile
from instaloader.exceptions import *
from itertools import dropwhile, takewhile
import time
from datetime import datetime, timedelta
import numpy as np

class InstaDataExtractor(object):
    
    def __init__(self):
        self.L = self.login()
        
    def login(self):
        L = Instaloader()
        return L
    
    def get_likes(self, hashtag = 'travel'):
        ''' default hashtag is travel 
            return 100 recent people usernames who liked a particular hashtag! 
        '''
        assert type(hashtag) == str
        L = self.L
        data = []
        posts = L.get_hashtag_posts(hashtag)
        likes = set()

        today = datetime.today()
        tomorrow = datetime.today() + timedelta(days = 2)
        yesterday = datetime.today() + timedelta(days = -1)

        # Since is big | less past 
        SINCE = tomorrow
        # Until is small | past
        UNTIL = yesterday

        # Extract the list of people to send request to, note : We are selecting the users who are recently active. 
        for post in takewhile(lambda p: p.date > UNTIL, dropwhile(lambda p: p.date > SINCE, posts)):
            likes = post.get_likes()
            for like in likes:
                data.append(like.username)
            users = np.unique(data)
            if len(users) >= 100: 
                break;
            else:
                pass
            
        return users[:100]

    def get_followers(self, user):
        """ returns list of followers of user """
        # Extracting list of my follower and followees !!
        L = self.L
        profile = Profile.from_username(L.context, user)
        followers = []
        for follower in profile.get_followers():
            followers.append(follower.username)
        return followers
        
    def get_followees(self, user):
        ''' returns list of followees of users '''
        L = self.L
        profile = Profile.from_username(L.context, user)
        followees = []
        for followee in profile.get_followees():
            followees.append(followee.username)
        return followees
        
    def get_post(self, user, path = ''):
      '''return all the post of the user'''
      L = self.L
      profile = Profile.from_username(L.context, user)
      for post in profile.get_posts():
        L.download_post(post, target= path)

    def users_data(self, users):
        ''' Take a list of users and return these data.
            
            'username', 'followers', 'followees', 'private_account', 'post_media', 'post_igtv', 'full_name', 'biography'
            
            Suggested to put 100 user at a time. 
        '''
        L = self.L
        data = []
        for count, user in enumerate(users):
            try: 
                profile = Profile.from_username(L.context, user)
                if count % 5 == 0:
                    time.sleep(1)
                followers = profile.followers
                followees = profile.followees
                private_account = profile.is_private
                posts_media = profile.mediacount
                posts_igtv = profile.igtvcount
                full_name = profile.full_name
                bio = profile.biography
                data.append((user, followers, followees, private_account, posts_media, posts_igtv, full_name, bio))
            except ProfileNotExistsException:
                pass
        return data

In [2]:
import pandas as pd
from datetime import datetime, timedelta

dataExtractor = InstaDataExtractor()


In [4]:
path = 'data/'
# hashtags = ['fcbarcelona', 'cairo', 'weedmemes']
# hashtags = ['malaga', 'granada', 'bilbao', 'gay', 'nigeria', 'blackmemes']
hashtags = ['meme', 'coronavirus', 'jesus']

# local hashtag for chile 
# 'chile', 
hashtags = [ 'santiago', 'chilegram', 'santiagodechile']
hashtags = ['italy', 'italia', 'juventus', 'roma', 'acmilan', 'intermilan', 'rome']
today = datetime.today()
for count, hashtag in enumerate(hashtags):
    print(hashtag)
    start = time.time()
    users = dataExtractor.get_likes(hashtag = 'photography')
    print(len(users))
    time.sleep(2)
    data = dataExtractor.users_data(users)
    print(len(data))
    columns = ['username', 'followers', 'followees', 'private_account', 'post_media', 'post_igtv', 'full_name', 'biography']
    dataframe = pd.DataFrame(data = data, columns = columns)
    dataframe['hashtag'] = hashtag
    dataframe['date'] = today.date()
    dataframe.to_csv(path + hashtag + str(count) + str(today.date()) + '.csv' , index = None)
    end = time.time()
    print(hashtag + str(count), end - start)

italy
100
100
italy0 333.87057638168335
italia
100

Too many queries in the last time. Need to wait 57 seconds, until 22:45.
100
italia1 413.34433221817017
juventus
100
100
juventus2 369.32834029197693
roma
100

Too many queries in the last time. Need to wait 11 seconds, until 22:58.

Too many queries in the last time. Need to wait 13 seconds, until 22:58.
100
roma3 387.84614300727844
acmilan


JSON Query to p/CANgEGgnPKb/: 404 Not Found [retrying; skip with ^C]
JSON Query to p/CANgEGgnPKb/: 404 Not Found [retrying; skip with ^C]


QueryReturnedNotFoundException: JSON Query to p/CANgEGgnPKb/: 404 Not Found